<a href="https://colab.research.google.com/github/SimranJHAA/CATDOGCNN/blob/main/CATDOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Define transformations for data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths to train and test folders in Google Drive
train_path = '/content/gdrive/MyDrive/train_data'
test_path = '/content/gdrive/MyDrive/test_data'

# Create datasets
train_dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet50 model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 2)
)

# Set device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Implementing learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5, min_lr=1e-7)

# Training loop with early stopping
num_epochs = 20
patience = 10
best_loss = float('inf')
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item()

    # Print average loss for the epoch
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Validate the model on test data
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(test_loader)
    print(f"Validation Loss: {val_loss:.4f}")

    # Update learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_loss:
        best_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

print('Finished Training')

# Load the best model for evaluation
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')


Epoch [1/20], Loss: 0.5358
Validation Loss: 42.5208
Epoch [2/20], Loss: 0.4596
Validation Loss: 25.4166
Epoch [3/20], Loss: 0.3997
Validation Loss: 0.4402
Epoch [4/20], Loss: 0.3190
Validation Loss: 0.7151
Epoch [5/20], Loss: 0.2909
Validation Loss: 0.9202
Epoch [6/20], Loss: 0.3110
Validation Loss: 1.0108
Epoch [7/20], Loss: 0.3242
Validation Loss: 0.4028
Epoch [8/20], Loss: 0.2877
Validation Loss: 0.4239
Epoch [9/20], Loss: 0.1723
Validation Loss: 0.5528
Epoch [10/20], Loss: 0.1914
Validation Loss: 0.5894
Epoch [11/20], Loss: 0.2178
Validation Loss: 1.3939
Epoch [12/20], Loss: 0.2443
Validation Loss: 0.8468
Epoch [13/20], Loss: 0.1517
Validation Loss: 0.4228
Epoch [14/20], Loss: 0.1030
Validation Loss: 0.3478
Epoch [15/20], Loss: 0.0587
Validation Loss: 0.3111
Epoch [16/20], Loss: 0.0599
Validation Loss: 0.3407
Epoch [17/20], Loss: 0.0594
Validation Loss: 0.3406
Epoch [18/20], Loss: 0.0510
Validation Loss: 0.3158
Epoch [19/20], Loss: 0.0453
Validation Loss: 0.3210
Epoch [20/20], Loss

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Define transformations for data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths to train and test folders in Google Drive
train_path = '/content/gdrive/MyDrive/train_data'
test_path = '/content/gdrive/MyDrive/test_data'

# Create datasets
train_dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet50 model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 2)
)

# Set device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Implementing learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5, min_lr=1e-7)

# Training loop with early stopping
num_epochs = 20
patience = 10
best_loss = float('inf')
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item()

    # Print average loss for the epoch
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Validate the model on test data
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(test_loader)
    print(f"Validation Loss: {val_loss:.4f}")

    # Update learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_loss:
        best_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

print('Finished Training')

# Load the best model for evaluation
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')


Epoch [1/20], Loss: 0.5358
Validation Loss: 42.5208
Epoch [2/20], Loss: 0.4596
Validation Loss: 25.4166
Epoch [3/20], Loss: 0.3997
Validation Loss: 0.4402
Epoch [4/20], Loss: 0.3190
Validation Loss: 0.7151
Epoch [5/20], Loss: 0.2909
Validation Loss: 0.9202
Epoch [6/20], Loss: 0.3110
Validation Loss: 1.0108
Epoch [7/20], Loss: 0.3242
Validation Loss: 0.4028
Epoch [8/20], Loss: 0.2877
Validation Loss: 0.4239
Epoch [9/20], Loss: 0.1723
Validation Loss: 0.5528
Epoch [10/20], Loss: 0.1914
Validation Loss: 0.5894
Epoch [11/20], Loss: 0.2178
Validation Loss: 1.3939
Epoch [12/20], Loss: 0.2443
Validation Loss: 0.8468
Epoch [13/20], Loss: 0.1517
Validation Loss: 0.4228
Epoch [14/20], Loss: 0.1030
Validation Loss: 0.3478
Epoch [15/20], Loss: 0.0587
Validation Loss: 0.3111
Epoch [16/20], Loss: 0.0599
Validation Loss: 0.3407
Epoch [17/20], Loss: 0.0594
Validation Loss: 0.3406
Epoch [18/20], Loss: 0.0510
Validation Loss: 0.3158
Epoch [19/20], Loss: 0.0453
Validation Loss: 0.3210
Epoch [20/20], Loss

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Define transformations for data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize images to 512x512
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths to train and test folders in Google Drive
train_path = '/content/gdrive/MyDrive/train_data'
test_path = '/content/gdrive/MyDrive/test_data'

# Create datasets
train_dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet50 model and modify it for binary classification
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 2)
)

# Set device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Implementing learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5, min_lr=1e-7)

# Training loop with early stopping
num_epochs = 20
patience = 10
best_loss = float('inf')
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item()


    # Print average loss and training accuracy for the epoch
    epoch_loss = running_loss / len(train_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Validate the model on test data
    model.eval()
    val_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()



    # Update learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_loss:
        best_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

print('Finished Training')

# Load the best model for evaluation
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Evaluate on test set
correct_final = 0
total_final = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted_final = torch.max(outputs.data, 1)
        total_final += labels.size(0)
        correct_final += (predicted_final == labels).sum().item()

final_test_accuracy = 100 * correct_final / total_final
print(f'Final Test Accuracy: {final_test_accuracy:.2f}%')


Epoch [1/20], Loss: 0.6793
Epoch [2/20], Loss: 0.6124
Epoch [3/20], Loss: 0.5477
Epoch [4/20], Loss: 0.4784
Epoch [5/20], Loss: 0.4510
Epoch [6/20], Loss: 0.4786
Epoch [7/20], Loss: 0.4513
Epoch [8/20], Loss: 0.4410
Epoch [9/20], Loss: 0.3503
Epoch [10/20], Loss: 0.3275
Epoch [11/20], Loss: 0.2912
Epoch [12/20], Loss: 0.2684
Epoch [13/20], Loss: 0.2251
Epoch [14/20], Loss: 0.2899
Epoch [15/20], Loss: 0.2437
Epoch [16/20], Loss: 0.2628
Epoch [17/20], Loss: 0.2867
Epoch [18/20], Loss: 0.2413
Epoch [19/20], Loss: 0.1532
Epoch [20/20], Loss: 0.1164
Finished Training
Final Test Accuracy: 85.00%
